* project 目錄為 `End-to-End-Machine-Learning-Pipeline`，以下路徑均以此為 根目錄

* `src/mlProject/constants/__init__.py` 下定義了 

	```python
	CONFIG_FILE_PATH 
	PARAMS_FILE_PATH 
	SCHEMA_FILE_PATH
	```

* `src/mlProject/constants/__init__.py` 要先完成 (針對 5. Update the configuration manager )

# 自行實作

In [ ]:
!git clone https://github.com/henrykohl/MLOps-Foundation.git

Cloning into 'MLOps-Foundation'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 252 (delta 100), reused 44 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (252/252), 2.99 MiB | 18.33 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
%cd MLOps-Foundation/EtoEMLPipeline/

/content/MLOps-Foundation/EtoEMLPipeline


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [ ]:
!conda create -n mlproj python=3.8 -y

In [ ]:
%pwd

'/content/MLOps-Foundation/EtoEMLPipeline'

In [ ]:
!source activate mlproj; pip install -r requirements.txt

In [ ]:
%%bash
source activate mlproj

python

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-10 11:45:45,038: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-12-10 11:45:45,041: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-10 11:45:45,044: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-10 11:45:45,045: INFO: common: created directory at: artifacts]
[2025-12-10 11:45:45,045: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-10 11:45:45,414: INFO: <stdin>: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4C98:D8880:7A131:A452B:69395D68
Accept-Ranges: bytes
Date: Wed, 10 Dec 2

# Lecture Demo

In [ ]:
import os
os.chdir("../")
%pwd

'd:\\Bappy\\Live Sessions\\Euron\\MLOPs Masters Batch\\End-to-End-Machine-Learning-Pipeline'

## 4. Update the entity
* project 對應 `src/mlProject/entity/config_entity.py`

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path           ## 實例化時，儲存的類型是 str
    source_URL: str
    local_data_file: Path    ## 實例化時，儲存的類型是 str; data.zip 路徑
    unzip_dir: Path          ## 實例化時，儲存的類型是 str

## 5. Update the configuration manager
* project 對應 `src/mlProject/config/configuration.py`用到
	- '4. entity'： `src/mlProject/entity/config_entity.py` -- 輸出 DataIngestionConfig

In [ ]:
## CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH

(PosixPath('config/config.yaml'),
 PosixPath('params.yaml'),
 PosixPath('schema.yaml'))

In [ ]:
## config_filepath = CONFIG_FILE_PATH
## params_filepath = PARAMS_FILE_PATH
## schema_filepath = SCHEMA_FILE_PATH

## config = read_yaml(config_filepath)
## params = read_yaml(params_filepath)
## schema = read_yaml(schema_filepath)

In [ ]:
## config

ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

In [ ]:
## params

ConfigBox({'ElasticNet': {'alpha': 0.2, 'l1_ratio': 0.1}})

In [ ]:
## schema

ConfigBox({'COLUMNS': {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}, 'TARGET_COLUMN': {'name': 'quality'}})

In [ ]:
## config.data_ingestion

ConfigBox({'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'})

---

In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,   ## 輸出: PosixPath("config/config.yaml")
        params_filepath = PARAMS_FILE_PATH,   ## 輸出: PosixPath("params.yaml")
        schema_filepath = SCHEMA_FILE_PATH):  ## 輸出: PosixPath("schema.yaml")

        self.config = read_yaml(config_filepath) ## 輸出: ConfigBox({...}); config.artifacts_root 是 str
        self.params = read_yaml(params_filepath) ## 輸出: ConfigBox({...})
        self.schema = read_yaml(schema_filepath) ## 輸出: ConfigBox({...})

        create_directories([self.config.artifacts_root]) ## 建立目錄 artifacts



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion  ## 輸出: ConfigBox({...}); 

        create_directories([config.root_dir]) ## 建目錄 artifacts/data_ingestion; config.root_dir 是 str

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,                ## config.root_dir 是 str
            source_URL=config.source_URL,            ## config.source_URL 是 str
            local_data_file=config.local_data_file,  ## config.local_data_file 是 str
            unzip_dir=config.unzip_dir               ## config.unzip_dir 是 str
        )

        return data_ingestion_config

## 6. update the components
* project 對應 `src/mlProject/components/data_ingestion.py` 用到
	- '4. entity'： `src/mlProject/entity/config_entity.py` -- 輸入 DataIngestionConfig

In [ ]:
## config.data_ingestion.local_data_file ## 測試

'artifacts/data_ingestion/data.zip'

In [ ]:
## os.path.exists(config.data_ingestion.local_data_file) ## 測試

False

In [ ]:
## filename, headers = request.urlretrieve(
##                 url = config.data_ingestion.source_URL,
##                 filename = config.data_ingestion.local_data_file
##             )

## filename, headers ## 測試

('artifacts/data_ingestion/data.zip',
 <http.client.HTTPMessage at 0x7ecdc516e240>)

In [ ]:
## config.data_ingestion.unzip_dir ## 測試

'artifacts/data_ingestion'

---

In [ ]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_file(self):
        if not os.path.exists(self.config.local_data_file): ## 檔案路徑 'artifacts/data_ingestion/data.zip'
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir    ## 解壓目錄路徑 'artifacts/data_ingestion'
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


## 7. Update the pipeline 

* project 對應 `src/mlProject/pipeline/stage_01_data_ingestion.py` 需調用
	- '5. configuration': `src/mlProject/config/configuration.py` -- 輸出 DataIngestionConfig
	- '6. components': `src/mlProject/components/data_ingestion.py` -- 輸入 DataIngestionConfig

* 完成後，建立 `artifacts/data_ingestion` 資料夾內包含 'data.zip' 與 'winequality-red.csv' 兩個檔案

In [ ]:
try:
    config = ConfigurationManager()                              ## 例化configuration，建立主目錄
    data_ingestion_config = config.get_data_ingestion_config()   ## 執行configuration，建立次目錄，例化entity
    data_ingestion = DataIngestion(config=data_ingestion_config) ## 例化 component
    data_ingestion.download_file()                               ## 執行 component，下載 zip 檔案
    data_ingestion.extract_zip_file()                            ## 執行 component，解壓 zip 檔案
except Exception as e:
    raise e

[2025-01-12 12:23:20,266: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-12 12:23:20,266: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-12 12:23:20,269: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-12 12:23:20,269: INFO: common: created directory at: artifacts]
[2025-01-12 12:23:20,269: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-12 12:23:21,833: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 756E:27C4B:282C52:3CE0D8:67835FD3
Accept-Ranges: bytes
Date: Sun, 12 